In [ ]:
!cp -r /content/drive/MyDrive/DATA/9.zip /content

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/content/8.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/DATA')


In [ ]:
import torch
import torchvision
from torchvision import datasets as dd
from torchvision import transforms as tt
from torch.utils.data import DataLoader,random_split
from torchvision.datasets import ImageFolder


In [ ]:
transform=tt.Compose([tt.ToTensor(),tt.Resize((60,60))])

In [ ]:
data=ImageFolder("/content/DATA", transform=transform)

In [ ]:
train_size=int(0.8*len(data))
val_size=len(data)-train_size

In [ ]:
val_size

42660

In [ ]:
train_dataset, val_dataset = random_split(data, [train_size, val_size])

In [ ]:
train_size

170636

In [ ]:
train_d=DataLoader(train_dataset,batch_size=32,shuffle=False)

In [ ]:
valid_d=DataLoader(val_dataset,batch_size=128,shuffle=False)

In [ ]:
import torch.nn as nn
import torchvision.models as models

In [ ]:
model=models.vgg16_bn(pretrained=False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model.classifier[6]=nn.Linear(in_features=4096, out_features=12)

In [ ]:
model

In [ ]:
model.to(device)

In [ ]:
from tqdm import tqdm

In [ ]:
import torch.nn.functional as F
import torch.optim as opt
lr=1e-4
num_epochs=2
optimizer=opt.Adam(model.parameters(),lr)
loss_f=nn.CrossEntropyLoss()



In [ ]:
cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:

check_dir='/content/drive/MyDrive/MODEL'
!cd /content/drive/MyDrive/MODEL

In [ ]:
pwd

'/content/drive/MyDrive'

In [ ]:
for epoch in range(num_epochs):
  losses=[]
  for batch_idx, (data,targets) in enumerate(tqdm(train_d)):
    data=data.to(device=device)
    targets=targets.to(device=device)
    scores=model(data)
    loss=loss_f(scores,targets)

    losses.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")
  checkpoint_file = f'{check_dir}checkpoint_epoch{epoch}.pt'
  checkpoint_file1 = f'{check_dir}checkpoint_epoch{epoch}.pt'
  torch.save(model.state_dict(),checkpoint_file)
  torch.save(model,checkpoint_file1)
def check_accuracy(loader, model):
    #if loader.dataset.train:
    #    print("Checking accuracy on training data")
    #else:
    #    print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


check_accuracy(valid_d, model)

  0%|          | 0/5333 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 5333/5333 [09:14<00:00,  9.61it/s]


Cost at epoch 0 is 0.05017


100%|██████████| 5333/5333 [09:14<00:00,  9.62it/s]


Cost at epoch 1 is 0.04067
Got 42357 / 42660 with accuracy 99.29


In [ ]:
modelpath="/content/drive/MyDrive/MODEL/Model_Check.pt"
torch.save(model,modelpath)